# Flask (продолжение)

## Шаблоны страниц

Чтобы сайт проще было конструировать, применяются шаблоны. Мы уже смотрели на Jinja templates и как можно подставлять значения переменных в страницы.

Можно соединять разные блоки в странице, чтобы не копировать части html-кода, который повторяется на всех или многих страницах. Например, в ВК есть стандартная панель сбоку, которая не меняется от страницы к странице и меняется только центральная полоса. Или на сайтах бывает боковое меню, верхнее меню, новостная лента, которые постоянны в шаблоне, а меняется только часть страницы.

Для этого создается базовый шаблон страницы (base.html), который потом просто дополняется информацией, которая отличается на страницах. Мы можем на ней загрузить наши стили, скрипты и прочие зависимости и не думать про это позже. 

Поместим сюда меню, так как оно не меняется, стили, добавим поля основной части страницы.

Мы также указываем блоки, куда потом вставим наши отличающиеся кусочки.

Для этого пропишем ```{% block title %} {% endblock %}``` и ```{% block content %} {% endblock %}```

title и content - это названия наших блоков, их содержимое пока пустое, мы его заменим.

```html
<!DOCTYPE html>
<html lang="en">
    
<head>
    <meta charset="UTF-8">
    <title>{% block title %} {% endblock %}</title>
         <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">

    <!-- Bootstrap CSS, JS-->
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@4.5.3/dist/css/bootstrap.min.css" integrity="sha384-TX8t27EcRE3e/ihU7zmQxVncDAy5uIKz4rEkgIXeMed4M0jlfIDPvg6uqKI2xXr2" crossorigin="anonymous">
    <script src="https://code.jquery.com/jquery-3.5.1.slim.min.js" integrity="sha384-DfXdz2htPH0lsSSs5nCTpuj/zy4C+OGpamoFVy38MVBnE+IbbVYUew+OrCXaRkfj" crossorigin="anonymous"></script>
    <script src="https://cdn.jsdelivr.net/npm/bootstrap@4.5.3/dist/js/bootstrap.bundle.min.js" integrity="sha384-ho+j7jyWK8fNQe+A12Hb8AhRq26LrZ/JpcUGGOn+Y7RsweNrtN/tE3MoK7ZeZDyx" crossorigin="anonymous"></script>
</head>
    
<body>
    <!-- вот отсюда https://getbootstrap.com/docs/4.5/components/navbar/ -->
    <nav class="navbar navbar-expand-lg navbar-light bg-light">
      <a class="navbar-brand" href="#">База фильмов</a>
      <button class="navbar-toggler" type="button" data-toggle="collapse" data-target="#navbarNavAltMarkup" aria-controls="navbarNavAltMarkup" aria-expanded="false" aria-label="Toggle navigation">
        <span class="navbar-toggler-icon"></span>
      </button>
      <div class="collapse navbar-collapse" id="navbarNavAltMarkup">
        <div class="navbar-nav">
          <a class="nav-link" href="/rating">Рейтинг фильмов</a>
          <a class="nav-link" href="/search">Поиск</a>
        </div>
      </div>
    </nav>

    <div style="margin: 50px 10%">
        {% block content %} {% endblock %}
    </div>


</body>
</html>
```

Теперь на главной странице просто подставим информацию.

Указываем, что мы этим кодом раширяем base.html. Вписываем блоки

```html
{% extends 'base.html' %}

{% block title %} База фильмов {% endblock %}

{% block content %}
    <h1>Текст главной страницы</h1>
{% endblock %}
```

## Запросы к базе и использование объектов

Можно импортировать наши модели (классы) и использовать их для запросов к базе. 

### Запрос по ключу

Самый простой пример - это запрос одного объекта по айди. Сделаем страницу для человека, url будет выгялдеть как ```http://127.0.0.1:5000/person/186505```. Мы получаем айди человека и просто в базе получаем информацию о человеке с таким ключом.

Используем метод get для получения по ключу. Обратно нам выдастся объект типа Person, мы можем использовать атрибуты, которые мы прописали в модели. Передаем этот объект на страницу и вписываем в шаблон переменные.

In [ ]:
@app.route("/person/<person_id>")
def person_page(person_id):
    person = Person.query.get(person_id)
    return render_template("person.html", person=person)

```html
{% extends 'base.html' %}

{% block title %} {{ person.name }}{% endblock %}

{% block content %}
    <table class="table">
        <tr>
            <td>Имя</td>
            <td>{{ person.name }}</td>
        </tr>
        <tr>
            <td>Год рождения</td>
            <td>{{ person.born }}</td>
        </tr>
        {% if person.died %}
        <tr>
            <td>Год смерти</td>
            <td>{{ person.died }}</td>
        </tr>
        {% endif %}
        <tr>
            <td>Фильмы</td>
            <td>
                {% for film in person.films %}
                    {{ film.title }} <br>
                {% endfor %}
            </td>
        </tr>
    </table>
{% endblock %}

```

### Топ фильмов

В запросах от класса мы не можем нормально искать по атрибутам, которые привязаны (например, список актеров к фильму), но мы можем сделать join.

Мы делаем запрос через db.session.query, в нашем случае это будет что-то вроде курсора. Мы запрашиваем таблицу Film, соединяем ее с Rating.

Теперь фильтруем. Добавляем фильтр, что число оценок больше 100 тысяч, сортируем по рейтингу по убыванию.

Берем топ-250. Получаем ответ с помощью all (как fetchall).

Мы получили список объектов типа Film, которые отсортированы и отфильтрованы как мы просили. Это удобнее, чем просто кортежи, где нам надо помнить, что на какой позиции лежит, мы просто тут можем обращаться по именам атрибутов.

In [ ]:
@app.route("/rating")
def rating():
    rating = db.session.query(Film)\
        .join(Rating)\
        .filter(Rating.votes > 100000)\
        .order_by(-Rating.value)\
        .limit(250).all()
    return render_template("rating.html", rating=rating)

В шаблоне страницы мы можем идти по объектам (как в обычном списке в питоне), но мы хотим помнить номер порядковый, поэтому попробуем идти по индексам, для это сделаем range от 0 до длины списка).

```html
{% extends 'base.html' %}

{% block title %} Рейтинг фильмов {% endblock %}

{% block content %}
    <table class="table">
        {% for i in range(0, rating|length) %}
            <tr>
                <td scope="row">{{ i + 1 }}</td>
                <td>{{ rating[i].rating.value }}</td>
                <td>{{ rating[i].premiered }}{% if rating[i].ended %}-{{ rating[i].ended }}{% endif %}</td>
                <td><a href="/film/{{ rating[i].film_id }}" target="_blank">{{ rating[i].title }}</a></td>
            </tr>
        {% endfor %}
    </table>
{% endblock %}
```

## Обработка форм


см. проект и [пост вот тут](https://code.luasoftware.com/tutorials/flask/flask-get-request-parameters-get-post-and-json/)

## Пример работы с анкетой


[пример](https://github.com/hse-ling-python/seminars/blob/master/flask_applications/untitled/app.py)

для сохранения данных можно воспользоваться классами таблиц баз данных

In [ ]:
@app.route('/process', methods=['get'])
def answer_process():
    # если нет ответов, то отсылаем решать анкету
    if not request.args:
        return redirect(url_for('question_page'))
    
    # достаем параметры
    gender = request.args.get('gender')
    education = request.args.get('education')
    age = request.args.get('age')
    
    # создаем профиль пользователя
    user = User(
        age=age,
        gender=gender,
        education=education
    )
    # добавляем в базу
    db.session.add(user)
    # сохраняемся
    db.session.commit()
    # получаем юзера с айди (автоинкремент)
    db.session.refresh(user)
    
    # получаем два ответа
    q1 = request.args.get('q1')
    q2 = request.args.get('q2')
    
    # привязываем к пользователю (см. модели в проекте)
    answer = Answers(id=user.id, q1=q1, q2=q2)
    # добавляем ответ в базу
    db.session.add(answer)
    # сохраняемся
    db.session.commit()
    
    return 'Ok'